In [14]:
!python -m ipykernel install --user --name daivo --display-name "Python (daivo)"

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Installed kernelspec daivo in /Users/daivo/Library/Jupyter/kernels/daivo


In [15]:
pip install yfinance requests vaderSentiment pandas matplotlib


Note: you may need to restart the kernel to use updated packages.


In [20]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def fetch_news(company="Apple", api_key="", days_back=7):
    to_date = datetime.today().date()
    from_date = to_date - timedelta(days=days_back)

    url = (
        f"https://newsapi.org/v2/everything?q={company}"
        f"&from={from_date}&to={to_date}&language=en&sortBy=publishedAt"
        f"&pageSize=100&apiKey={api_key}"
    )
    
    response = requests.get(url)
    if response.status_code != 200:
        print("❌ Error:", response.status_code, response.json())
        return None

    articles = response.json().get("articles", [])
    df = pd.DataFrame([{
        "publishedAt": a["publishedAt"],
        "title": a["title"],
        "description": a["description"],
        "url": a["url"]
    } for a in articles])

    file_name = f"../data/sentiment_data/news_{company.lower()}.csv"
    df.to_csv(file_name, index=False)
    print(f"✅ Saved {len(df)} articles about {company} to {file_name}")
    return df


In [21]:
# Replace with your real NewsAPI key!
news_df = fetch_news(company="Apple", api_key="836b0aaf90e74987a9d5d5eb9296570a")
news_df.head()

✅ Saved 95 articles about Apple to ../data/sentiment_data/news_apple.csv


,publishedAt,title,description,url
0,2025-08-05T17:45:24Z,You’re more likely to win Powerball 100 times ...,A California mother's kids were all born on th...,https://www.bostonherald.com/2025/08/05/califo...
1,2025-08-05T17:45:24Z,You’re more likely to win Powerball 100 times ...,A California mother's kids were all born on th...,https://www.denverpost.com/2025/08/05/californ...
2,2025-08-05T17:44:34Z,Apple Is Testing AI Chatbot Support Assistant ...,Apple is quietly testing a ChatGPT-style Suppo...,https://www.redmondpie.com/apple-is-testing-ai...
3,2025-08-05T17:43:01Z,New FOX One streaming service launches August ...,Fox Corporation on Tuesday announced that FOX ...,https://macdailynews.com/2025/08/05/new-fox-on...
4,2025-08-05T17:42:21Z,This favorite ad blocker finally comes to Safa...,Fans of ad-blocking browser extensions will be...,https://www.zdnet.com/article/this-favorite-ad...


## Fetch Stock From yfinance

In [32]:
pip install alpha_vantage


Note: you may need to restart the kernel to use updated packages.


In [36]:
import requests
import pandas as pd
import os

def fetch_alpha_vantage_data(symbol="AAPL", api_key="YOUR_API_KEY"):
    url = (
        f"https://www.alphavantage.co/query?"
        f"function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=compact"
        f"&datatype=csv&apikey={api_key}"
    )

    response = requests.get(url)
    if response.status_code != 200 or "timestamp" not in response.text:
        print("❌ Error fetching data from Alpha Vantage.")
        return None

    os.makedirs("../data/price_data", exist_ok=True)
    file_path = f"../data/price_data/{symbol}.csv"

    with open(file_path, "w") as f:
        f.write(response.text)

    df = pd.read_csv(file_path, parse_dates=["timestamp"])
    df.rename(columns={"timestamp": "Date"}, inplace=True)
    df.sort_values("Date", inplace=True)
    print(f"✅ Saved Alpha Vantage {symbol} data to {file_path}")
    return df

# Example usage
df = fetch_alpha_vantage_data(symbol="AAPL", api_key="9GZT7PME36OPSTHO")
if df is not None:
    display(df.head())


✅ Saved Alpha Vantage AAPL data to ../data/price_data/AAPL.csv


,Date,open,high,low,close,volume
99,2025-03-13,215.95,216.8394,208.42,209.68,61368330
98,2025-03-14,211.25,213.9500,209.58,213.49,60107582
97,2025-03-17,213.31,215.2200,209.97,214.00,48073426
96,2025-03-18,214.16,215.1500,211.49,212.69,42432426
95,2025-03-19,214.22,218.7600,213.75,215.24,54385391
